# ID2209 Final Project - Vickery Auction

Number of bidders is set by changing numbidders variable





In [ ]:
!pip install pyautogen
import autogen
config_list_gpt_35 = [
  {
    "model": "gpt-3.5-turbo",
    "api_version": "2023-03-01-preview",
    "api_key": "<OPEN_AI_API_KEY>",
  }
]

#Construct Agents

In [ ]:
llm_config = {"config_list": config_list_gpt_35, "cache_seed": 42}

In [ ]:
bidder_names = [] #list of bidder names
numbidders = 10 #number of bidders who'd like to participate in an auction
bidders = [] #list of bidders


#creates number of bidders specified by user and gives them instructions
for i in range(numbidders):
  bidder_names.append("bidder" + str(i))
  bidders.append(autogen.AssistantAgent(
    name=bidder_names[i],
    system_message = """Your name is bidder""" + str(i) + """and you are participating in a vickery auction as a bidder.
The auction is conducted by {auction_agent}.
You should start with a randomly generated bid value in the range of 500-1000 and send it to the auctioneer when requested.
Do not include extra chit-chat
""",
    llm_config=llm_config,
    max_consecutive_auto_reply = 1
  ))

#creates n auction_agent to run the auction
auction_agent = autogen.ConversableAgent(
    name="AUCTIONEER",
    system_message=""""You are an AI-powered Auctioneer.
You cooridinate a Vickery auction.
You receive bids from the different bidders.
The winner is the agent with the higest bid and the amount they must pay is the second highest bid.
when asked who won the auction return the answer in this format: Winner_name, int(amount).
If there is a problem just return the word error and explain why there was an error.""",
    llm_config=llm_config,
    max_consecutive_auto_reply = 3
)

#creates a human admin for 'administrative' questions
user_proxy = autogen.UserProxyAgent(
   name="User_proxy",
   system_message="A human admin.",
   human_input_mode="NEVER"
)

#Single Round FIPA English Auction Interaction Protocol
Initiator is the auctioneer

Participants are the bidders interested in corresponding auction

The auctioneer starts the auction and calls for proposals

http://www.fipa.org/specs/fipa00031/XC00031F.html

https://ceur-ws.org/Vol-627/lads_1.pdf

In [ ]:
auction_agent.clear_history()

bids = []
opening_message = "I'm starting a new Vickery auction for a valuable painting. Please send me your bid and your name."
for i in range(numbidders):
  bidders[i].clear_history()
  auction_agent.send(
    message = opening_message,
    recipient = bidders[i],
    request_reply= True
  )
  bids.append(bidders[i].last_message())



AUCTIONEER (to bidder0):

I'm starting a new Vickery auction for a valuable painting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bidder0 (to AUCTIONEER):

My name is bidder0 and I would like to participate in the Vickery auction for the valuable painting. My initial bid is $785.

--------------------------------------------------------------------------------
AUCTIONEER (to bidder1):

I'm starting a new Vickery auction for a valuable painting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bidder1 (to AUCTIONEER):

My bid is 825. My name is bidder1.

--------------------------------------------------------------------------------
AUCTIONEER (to bidder2):

I'm starting a new Vickery auction for a valuable painting. Please send me your bid and your name.

--------------------------------------------------------------------------------
bid

#Bid summarization
This section is just for clarity

In [ ]:
#Asks each auctioneer to summarize their bids (just for clarity)

auction_agent.clear_history()
user_proxy.clear_history()
instruction_message = "The bids received were "+ str(bids) + "\nPlease summarize the bidders and their bids in a clear list"
user_proxy.send(
  message = instruction_message,
  recipient = auction_agent,
  request_reply = True)

User_proxy (to AUCTIONEER):

The bids received were [{'content': 'My name is bidder0 and I would like to participate in the Vickery auction for the valuable painting. My initial bid is $785.', 'role': 'assistant'}, {'content': 'My bid is 825. My name is bidder1.', 'role': 'assistant'}, {'content': 'My name is bidder2 and my bid for the valuable painting is $753.', 'role': 'assistant'}, {'content': 'My bid for the painting is 765. My name is bidder3.', 'role': 'assistant'}, {'content': 'My bid is 780 and my name is bidder4.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is $782. My name is bidder5.', 'role': 'assistant'}, {'content': 'My bid is 785 and my name is bidder6.', 'role': 'assistant'}, {'content': 'My bid is 742 and my name is bidder7.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is 802.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is $720. My name is bidder9.', 'role': 'assistant'}]
Please summarize the 

#Single Round FIPA English Auction Interaction Protocol (continued)
Auctioneer accepts the winning proposal and rejects all others

Informs winner of their win and how much they owe

Informs losers the auction is over

In [ ]:
#asks auctioneer to pick winner, display results, inform winner what they owe and the losers that the auction is over
user_proxy.send(

    message = "The bids received were "+ str(bids) + "Who had the highest bid? And what was the second highest bid? Return the winner and the second highest bid which is the amount they'll need to pay in this format: (Winner_name, int(amount_to_pay)). Provide only the answer",
    recipient = auction_agent,
    request_reply = True)
winning_message_info = auction_agent.last_message(user_proxy)
winning_message_info = winning_message_info['content']
value = winning_message_info
if(value == 'error'):
  print('There was an error, please try again')
else:
  # Remove parentheses and split the string
  value = value.strip('()')
  bidder, payment = value.split(', ')

  # Strip any extra spaces around the values
  bidder = bidder.strip()
  payment = payment.strip()

  # Extracted bidder and payment
  extracted_bidder = bidder  # Replace this with the extracted bidder variable

  # Notify the winner
  if extracted_bidder in bidder_names:
      print('HERE')
      winner_index = bidder_names.index(extracted_bidder)
      auction_agent.send(
        message = "You won the auction. You must pay $" + payment,
        recipient = bidders[winner_index],
        request_reply= True
      )

  # Notify the rest of the bidders
  for index, bidder in enumerate(bidder_names):
      if index != winner_index:
        auction_agent.send(
        message = "The auction is over. You have not won",
        recipient = bidders[index],
        request_reply= False
        )

User_proxy (to AUCTIONEER):

The bids received were [{'content': 'My name is bidder0 and I would like to participate in the Vickery auction for the valuable painting. My initial bid is $785.', 'role': 'assistant'}, {'content': 'My bid is 825. My name is bidder1.', 'role': 'assistant'}, {'content': 'My name is bidder2 and my bid for the valuable painting is $753.', 'role': 'assistant'}, {'content': 'My bid for the painting is 765. My name is bidder3.', 'role': 'assistant'}, {'content': 'My bid is 780 and my name is bidder4.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is $782. My name is bidder5.', 'role': 'assistant'}, {'content': 'My bid is 785 and my name is bidder6.', 'role': 'assistant'}, {'content': 'My bid is 742 and my name is bidder7.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is 802.', 'role': 'assistant'}, {'content': 'My bid for the valuable painting is $720. My name is bidder9.', 'role': 'assistant'}]Who had the highest bi